# 2.0 — Vector Search for the Book Recommender Project  
<img src="./378a3492-bb5e-445e-8ca8-6b36d0678d41.png" width="250" height="300">  

In this notebook, we will build the **vector search component** of our **semantic book recommender system**.  
The goal is to **convert book descriptions into vector embeddings, store them in a vector database, and enable semantic search** to recommend books based on meaning rather than simple keyword matches.  

# Dataset Link  

We will use the **cleaned books dataset** prepared in the previous data exploration notebook.  
It contains book titles, descriptions, categories, and optional emotion scores:  
[📚 Click here to view/download the dataset on Kaggle](https://www.kaggle.com/datasets/dylanjcastillo/7k-books-with-metadata)  

### What we will do in this notebook:  

- Generate vector embeddings for book descriptions using HuggingFace embedding models.  
- Create a vector database with **ChromaDB** to store embeddings and metadata.  
- Perform semantic search to retrieve books relevant to a user query.  
- Test the search with sample queries and analyze results.  
- Lay the groundwork for integrating this vector search into a **user-facing recommender system**.  

## Table of Contents  

- [1 - Packages](#1)  
- [2 - Loading the Dataset](#2)  
- [3 - Preparing Embeddings](#3)  
- [4 - Creating the Vector Database](#4)  
- [5 - Performing Semantic Search](#5)  
- [6 - Testing and Evaluation](#6)  





## 1 - Packages <a id="1"></a>


Before diving into building our vector search, let's explore  present our envirement with our first framework: the **LangChain framework**, which simplifies the development of applications using large language models (LLMs).

###  What is LangChain?

LangChain is an open-source Python framework designed to facilitate the creation of applications powered by LLMs. It provides modular components that enable developers to:

- Load and process documents.
- Split text into manageable chunks.
- Generate embeddings using various models.
- Store and retrieve embeddings efficiently using vector databases.

###  Core Components

LangChain's architecture comprises several key components:

- **Document Loaders**: Tools to load documents from various sources.  
- **Text Splitters**: Utilities to divide large documents into smaller chunks.  
- **Embeddings**: Methods to convert text into vector representations.  
- **Vector Stores**: Databases to store and retrieve embeddings.  
- **Chains**: Sequences of operations that process data.  
- **Agents**: Systems that use LLMs to decide the control flow of an application.  
- **Memory**: Mechanisms to maintain state across interactions.

These components can be combined to build complex workflows for tasks like semantic search, summarization, and question answering.

### 🖼 LangChain Architecture Diagram

<img src="./langchain.png">  

This diagram illustrates the hierarchical organization of the LangChain framework, showing how its components interact across multiple layers.

###  Similar Frameworks

Other frameworks offer similar functionalities:

- **LangGraph**: Modular framework for scalable, visualized, big data ML workflows.  
- **LlamaIndex**: Facilitates creation of indices over large datasets for efficient retrieval.  

These alternatives may be better suited for specific project requirements, but LangChain provides a comprehensive, modular approach for LLM-based applications.


<img src="./34e0149e-3bd2-4d8f-a88b-1ead4e5a4e17.png" width="500" height="300">

## - Understanding Chroma and Vector Databases 

To perform semantic search, we need a **vector database**. In our project, we use **Chroma**, which integrates seamlessly with LangChain.

###  What is Chroma?

Chroma is an open-source **vector database** designed to store and query embeddings efficiently.  
It allows us to:

- Persist embeddings and metadata (like book titles and categories).  
- Retrieve the most similar embeddings quickly using similarity search.  
- Integrate easily with language models via frameworks like LangChain.

###  What is a Vector Database?

A **vector database** is a database that stores **vector embeddings**—numerical representations of data (text, images, audio, etc.)—instead of traditional structured data.  

- Each piece of data is converted into a high-dimensional vector using an embedding model (e.g., HuggingFace or OpenAI).  
- Similarity between items is measured by comparing their vectors, often using **cosine similarity** or **Euclidean distance**.  

###  How Does a Vector Database Work?

1. **Embedding Generation**:  
   Text data (like book descriptions) is converted into vector embeddings. Each embedding is a list of numbers representing the semantic meaning of the text.

2. **Storage**:  
   Vectors are stored in the vector database along with metadata, e.g., title, category, or author.

3. **Querying / Similarity Search**:  
   - When a user submits a query, it is also converted into a vector.  
   - The database calculates similarity scores between the query vector and all stored vectors.  
   - The most similar items are returned as results.

4. **Ranking**:  
   Results are ranked by similarity score, giving the user the most relevant recommendations.

<img src="./vectordatabase (1).png" width="500" height="300">

###  Why Use a Vector Database?

- Traditional databases cannot efficiently perform **semantic search**.  
- Vector databases are optimized for **high-dimensional vector operations**, enabling real-time, context-aware search.  
- They are essential for modern recommender systems, AI search engines, and any application using embeddings.



This explanation prepares the user to understand **why we use Chroma with LangChain** and how semantic search works under the hood.


In [ ]:
! pip install langchain langchain-community langchain-text-splitters langchain-huggingface langchain-chroma


In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

## 2 - Loading the dataset <a id="2"></a>
Let's load the cleaned dataset generated from our previous data exploration.


In [6]:
import pandas as pd

books = pd.read_csv("books_cleaned.csv")

In [7]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine..."
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,Mistaken Identity,9788172235222 On A Train Journey Home To North...
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014 This book tells the tale of a ma...
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...


In [8]:
books["tagged_description"]

0       9780002005883 A NOVEL THAT READERS and critics...
1       9780002261982 A new 'Christie for Christmas' -...
2       9780006178736 A memorable, mesmerizing heroine...
3       9780006280897 Lewis' work on the nature of lov...
4       9780006280934 "In The Problem of Pain, C.S. Le...
                              ...                        
5192    9788172235222 On A Train Journey Home To North...
5193    9788173031014 This book tells the tale of a ma...
5194    9788179921623 Wisdom to Create a Life of Passi...
5195    9788185300535 This collection of the timeless ...
5196    9789027712059 Since the three volume edition o...
Name: tagged_description, Length: 5197, dtype: object

In [9]:
books["tagged_description"].to_csv("tagged_description.txt",
                                   sep = "\n",
                                   index = False,
                                   header = False)

## 3 - Preparing Embeddings <a id="3"></a>
## 6 - Text Chunking with Sliding Window <a id="6"></a>

When preparing text for embeddings, it's important to split large documents into smaller chunks. This allows the embedding model to handle the text efficiently while preserving semantic meaning.

###  What is Chunking?

Chunking divides a large text into smaller, manageable segments called **chunks**. Each chunk can then be embedded separately, making semantic search feasible for long texts.

###  Sliding Window Concept

A **sliding window** is a technique where a fixed-size window moves across the text, creating overlapping chunks.  

- **Window Size (Chunk Size)**: Number of characters or tokens per chunk.  
- **Overlap**: Number of characters or tokens shared between consecutive chunks.  

Example with `chunk_size = 500` and `chunk_overlap = 50`:

- Chunk 1 → characters 0–499  
- Chunk 2 → characters 450–949  
- Chunk 3 → characters 900–1399  

This overlap ensures **context is preserved** across chunks.

###  Why Use Sliding Window?

- Preserves context at chunk boundaries.  
- Improves retrieval accuracy for semantic search.  
- Balances efficiency and meaningful embeddings.

###  Choosing Chunk Size and Overlap

- **Chunk Size**:
  - Should fit within your embedding model's input limit.  
  - Large enough to contain meaningful context, small enough for efficiency.  
  - Typical: 200–1000 characters (we use 500).  

- **Overlap**:
  - Preserves context between chunks.  
  - Typical: 10–20% of chunk size (we use 50, i.e., 10% of 500).  

- **Rule of Thumb**: Start with a chunk size roughly equal to paragraph length and 10–20% overlap, then adjust based on search performance.

### 🖼 Visual Example

![Sliding Window Chunking](https://bitpeak.com/wp-content/uploads/2023/04/Sliding-Window-Chunking-Example.png)

- Blue boxes = chunks  
- Overlap ensures semantic meaning at boundaries is retained  
- Adjusting window size and overlap changes number of chunks and retrieval accuracy



In [ ]:
# Step 1: Load the raw text documents from a file
# "tagged_description.txt" contains all book descriptions we want to embed

# raw_documents is now a list of Document objects, each containing text and optional metadata

# Step 2: Initialize a text splitter
# We want to split long documents into smaller chunks that are easier to embed
# chunk_size = 500 → each chunk will have up to 500 characters
# chunk_overlap = 50 → consecutive chunks will overlap by 50 characters for context
# separator = "\n" → split at newline characters when possible


In [11]:
raw_documents = TextLoader("tagged_description.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50, separator="\n")
documents = text_splitter.split_documents(raw_documents)

Created a chunk of size 1168, which is longer than the specified 500
Created a chunk of size 1214, which is longer than the specified 500
Created a chunk of size 960, which is longer than the specified 500
Created a chunk of size 843, which is longer than the specified 500
Created a chunk of size 881, which is longer than the specified 500
Created a chunk of size 1088, which is longer than the specified 500
Created a chunk of size 1189, which is longer than the specified 500
Created a chunk of size 513, which is longer than the specified 500
Created a chunk of size 752, which is longer than the specified 500
Created a chunk of size 728, which is longer than the specified 500
Created a chunk of size 721, which is longer than the specified 500
Created a chunk of size 1267, which is longer than the specified 500
Created a chunk of size 681, which is longer than the specified 500
Created a chunk of size 553, which is longer than the specified 500
Created a chunk of size 521, which is longe

In [12]:
documents[0]

Document(metadata={'source': 'tagged_description.txt'}, page_content='9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gi

In [14]:
import os
os.environ["USE_TF"] = "0"


In [17]:
import os
os.environ["USE_TF"] = "0"
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"


## 7 - Understanding Text Embeddings 

To build a semantic search system, the core concept is **text embeddings**.  

###  What is a Text Embedding?

- A **text embedding** is a **numerical vector** that represents the semantic meaning of a piece of text.  
- Instead of just comparing keywords, embeddings capture **context and meaning**, so similar concepts are close together in a high-dimensional space.  
- For example:
  - “A thrilling detective story”  
  - “A suspenseful mystery novel”  
  → These two sentences will have embeddings that are close in vector space, even if the words differ.

###  Why Use Embeddings?

- Enable **semantic search**: find text based on meaning, not exact words.  
- Support **recommendation systems**: similar books or documents can be retrieved easily.  
- Useful for **clustering and classification** tasks in NLP.  

### 🔹 Choosing an Embedding Model

1. **HuggingFace Embeddings**:
   - Open-source models, run locally or on your server.  
   - Example: `sentence-transformers/all-MiniLM-L6-v2`  
     - Lightweight and fast.  
     - Optimized for semantic similarity tasks.  
     - HuggingFace link: [all-MiniLM-L6-v2](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2)  
   - To choose a model:
     - Check task: semantic search, classification, clustering.  
     - Consider speed vs accuracy trade-off: smaller models → faster, larger models → more precise.  

2. **OpenAI Embeddings**:
   - Cloud-based API embeddings, e.g., `text-embedding-3-small` or `text-embedding-3-large`.  
   - High-quality embeddings suitable for production-scale semantic search.  
   - Example link: [OpenAI Embeddings](https://platform.openai.com/docs/guides/embeddings)  

3. **Other Options**:
   - Train custom embeddings with PyTorch or TensorFlow.  
   - Use domain-specific pre-trained models if your data is specialized.  

### 🔹 How Embeddings Work

- The embedding model converts each text chunk into a **vector of numbers**.  
- **Distance or similarity** between vectors (cosine similarity, Euclidean distance) measures semantic closeness.  
- This allows the vector database to **find the most relevant documents** for a query.



 **Key Takeaways**:

- Embeddings are the foundation of semantic search and recommendation systems.  
- Model choice depends on **accuracy requirements**, **speed**, and **deployment constraints**.  
- HuggingFace models like `all-MiniLM-L6-v2` are a great starting point for most semantic search tasks.

<img src="./similaritysearch.png" width="350" height="600">


In [ ]:
# Step 1: Provide the list of text chunks (`documents`) that need to be converted into embeddings.
# Step 2: Specify the embedding model using `HuggingFaceEmbeddings` with the pre-trained model `sentence-transformers/all-MiniLM-L6-v2`, which converts each text chunk into a high-dimensional vector capturing semantic meaning.
# Step 3: Assign a `collection_name` (`"books"`) in Chroma to store the embeddings and associated metadata.
# Step 4: The resulting `db_books` contains all the embeddings in the Chroma vector database, ready for semantic search queries.


## 4 - Creating the Vector Database <a id="4"></a>


In [16]:
db_books = Chroma.from_documents(
    documents,
    embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"),
    collection_name="books"
)

c:\Users\MatenTech\AppData\Local\anaconda3\envs\bd\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\MatenTech\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not insta

In [18]:
query = "A book to teach children about nature"
docs = db_books.similarity_search(query, k = 10)
docs

[Document(id='838811bf-1ace-4a1f-a25f-641da36385a5', metadata={'source': 'tagged_description.txt'}, page_content='9780786808069 Children will discover the exciting world of their own backyard in this introduction to familiar animals from cats and dogs to bugs and frogs. The combination of photographs, illustrations, and fun facts make this an accessible and delightful learning experience.'),
 Document(id='7e5f5a44-152a-42ba-9cd3-ae1acaabc99e', metadata={'source': 'tagged_description.txt'}, page_content="9780786808380 Introduce your babies to birds, cats, dogs, and babies through fine art, illustration, and photographs. These books are a rare opportunity to expose little ones to a range of images on a single subject, from simple child's drawings and abstract art to playful photos. A brief text accompanies each image, introducing the baby to some basic -- and sometimes playful -- information about the subjects."),
 Document(id='fb74564e-f675-40dd-9a3d-a9e068352d9c', metadata={'source': '

In [19]:
books[books["isbn13"] == int(docs[0].page_content.split()[0].strip())]

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
3747,9780786808069,0786808063,Baby Einstein: Neighborhood Animals,Marilyn Singer;Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=X9a4P...,Children will discover the exciting world of t...,2001.0,3.89,16.0,180.0,Baby Einstein: Neighborhood Animals,9780786808069 Children will discover the excit...


## 5 - Performing Semantic Search<a id="5"></a>



## 9 - Theory: Retrieval in Semantic Search and RAG <a id="9"></a>

### 🔹 What is Retrieval?

- **Retrieval** refers to the process of searching a dataset or database to find items that are **most relevant** to a given query.  
- In the context of semantic search:
  - Each document (or chunk) is converted into a **vector embedding**.
  - A query is also converted into a **vector embedding**.
  - The system finds documents whose vectors are **closest to the query vector** using similarity measures like **cosine similarity** or **Euclidean distance**.  

- Retrieval ensures that you can **access relevant knowledge** efficiently without scanning the entire dataset manually.

---

### 🔹 What is Retrieval-Augmented Generation (RAG)?

- **RAG** combines **retrieval** and **generation**:
  1. **Retrieve**: Find relevant documents or knowledge pieces from a vector database.  
  2. **Augment**: Feed the retrieved documents as context to a **language model**.  
  3. **Generate**: The language model produces a **response or answer** using the retrieved information.  

- Example use cases:
  - Answering questions using company documentation.
  - Providing book recommendations with generated summaries.
  - Chatbots that respond with accurate knowledge from large datasets.

---

### 🔹 How Semantic Search Fits In

- The `retrieve_semantic_recommendations` function we implemented:
  - Converts the user's query into a vector.
  - Retrieves the **most semantically similar book chunks**.
  - Returns relevant books from the database.
-  This is **retrieval**.
-  It is **not full RAG**, because:
  - There is **no language model generating new text**.
  - The output is only filtered **existing data** (books), not generated content.



###  Key Takeaways

- Retrieval is the **first step in RAG systems**.  
- Vector embeddings + similarity search allow **context-aware retrieval**.  
- RAG enhances retrieval by **feeding retrieved context into a generative model** to produce informative, natural language answers.  


In [ ]:
# Step 1: Define a function `retrieve_semantic_recommendations` that takes:
#   - `query`: a string representing the user's search or question
#   - `top_k`: the number of recommendations to return (default is 10)

# Step 2: Perform a similarity search in the Chroma vector database (`db_books`) using the query.
#   - `db_books.similarity_search(query, k=50)` finds the 50 most semantically similar document chunks to the query.
#   - The result, `recs`, is a list of Document objects containing text and metadata.

# Step 3: Initialize an empty list `books_list` to store ISBNs of recommended books.

# Step 4: Loop through each retrieved document chunk (`recs`) and extract the ISBN:
#   - `recs[i].page_content` accesses the text content of the chunk.
#   - `strip('"')` removes quotation marks.
#   - `split()[0]` extracts the first word, which contains the ISBN.
#   - Convert it to `int` and append to `books_list`.

# Step 5: Filter the original `books` DataFrame to return only rows whose ISBN (`isbn13`) is in `books_list`.

# Step 6: Return the filtered DataFrame containing the recommended books.


In [20]:
def retrieve_semantic_recommendations(
        query: str,
        top_k: int = 10,
) -> pd.DataFrame:
    recs = db_books.similarity_search(query, k = 50)

    books_list = []

    for i in range(0, len(recs)):
        books_list += [int(recs[i].page_content.strip('"').split()[0])]

    return books[books["isbn13"].isin(books_list)]

## 6 - Testing and Evaluation<a id="6"></a>


In [21]:
retrieve_semantic_recommendations("A book to teach children about nature")

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
31,9780007105045,0007105045,Tree and Leaf,John Ronald Reuel Tolkien,Literary Collections,http://books.google.com/books/content?id=aPb_A...,"""The two works 'On fairy-stories' and 'Leaf by...",2001.0,4.09,176.0,2245.0,Tree and Leaf: The Homecoming of Beorhtnoth : ...,"9780007105045 ""The two works 'On fairy-stories..."
59,9780007151240,0007151241,The Family Way,Tony Parsons,Parenthood,http://books.google.com/books/content?id=dJEIx...,It should be the most natural thing in the wor...,2005.0,3.51,400.0,2095.0,The Family Way,9780007151240 It should be the most natural th...
267,9780060882600,0060882603,The Annotated Charlotte's Web,E. B. White,Juvenile Nonfiction,http://books.google.com/books/content?id=vaYYH...,"Charlotte's Web, one of America's best-loved c...",2006.0,4.16,320.0,41.0,The Annotated Charlotte's Web,"9780060882600 Charlotte's Web, one of America'..."
324,9780060959036,0060959037,Prodigal Summer,Barbara Kingsolver,Fiction,http://books.google.com/books/content?id=06IwG...,Barbara Kingsolver's fifth novel is a hymn to ...,2001.0,4.00,444.0,85440.0,Prodigal Summer: A Novel,9780060959036 Barbara Kingsolver's fifth novel...
383,9780061144899,0061144894,When the Heart Waits,Sue Monk Kidd,Religion,http://books.google.com/books/content?id=JlP91...,From the Bestselling Author of The Secret Life...,2006.0,4.17,240.0,2141.0,When the Heart Waits: Spiritual Direction for ...,9780061144899 From the Bestselling Author of T...
404,9780064402453,0064402452,Racso and the Rats of NIMH,Jane Leslie Conly,Juvenile Fiction,http://books.google.com/books/content?id=MgoNv...,"‘Racso, a brash and boastful little rodent, is...",1988.0,3.76,288.0,3231.0,Racso and the Rats of NIMH,"9780064402453 ‘Racso, a brash and boastful lit..."
406,9780064403870,0064403874,"R-T, Margaret, and the Rats of NIMH",Jane Leslie Conly,Juvenile Fiction,http://books.google.com/books/content?id=WTHHH...,"When Margaret and her younger brother, Artie, ...",1991.0,3.52,272.0,631.0,"R-T, Margaret, and the Rats of NIMH",9780064403870 When Margaret and her younger br...
407,9780064404419,0064404412,The Rainbow People,Laurence Yep,Juvenile Fiction,http://books.google.com/books/content?id=5AHwq...,"""Culled from 69 stories collected in a [1930s]...",1992.0,3.75,208.0,202.0,The Rainbow People,"9780064404419 ""Culled from 69 stories collecte..."
416,9780064406925,006440692X,Winter on the Farm,Laura Ingalls Wilder,Juvenile Fiction,http://books.google.com/books/content?id=IvlKH...,The Little House books tell the story of a lit...,1997.0,4.13,32.0,400.0,Winter on the Farm,9780064406925 The Little House books tell the ...
427,9780064434874,0064434877,Christmas in the Big Woods,Laura Ingalls Wilder,Juvenile Fiction,http://books.google.com/books/content?id=FT1Yp...,"Long ago, a little girl named Laura Ingalls li...",1997.0,4.19,32.0,2062.0,Christmas in the Big Woods,"9780064434874 Long ago, a little girl named La..."


## 10 - Understanding the Vector Database Storage <a id="10"></a>

###  How Vector Databases Work

- A **vector database** (like Chroma, Pinecone, Weaviate, or FAISS) stores **vector embeddings** instead of traditional rows of text.  
- Each entry typically contains:
  1. The **embedding vector** (numerical representation of the text).  
  2. Optional **metadata** (like ISBN, title, author).  
  3. Optionally, the **original text chunk**.

- When you query the database:
  - The query is converted into a vector.
  - The database finds the **closest vectors** using similarity measures like **cosine similarity** or **Euclidean distance**.
  - Returns the associated metadata or original text.


###  Where is the Data Stored?

- **In-memory**: Some vector databases can run fully in memory for fast access (Chroma can do this).  
- **On-disk**: Others store data on disk for persistence (Chroma, FAISS with disk-backed index).  
- **Cloud storage**: Managed services like Pinecone, Weaviate, or Milvus store vectors in the cloud.  

- In the case of **Chroma**:
  - By default, it stores data locally in a folder named `chroma_db` (or a path you specify).  
  - This folder contains all embeddings, metadata, and collection info.
  - You can **inspect this folder**, but the data is typically in a serialized format, not human-readable.



###  Can You See the Stored Data?

- **Directly browsing**: You can see some metadata in JSON or SQLite-like files if Chroma is configured with a persistent directory.  
- **Programmatically**: Use Chroma API methods to query the database and inspect embeddings or metadata. Example:
  ```python
  db_books.get()


## 🎉 Congratulations!

You have successfully:

- Loaded and explored a dataset of books.  
- Split text into meaningful chunks using a sliding window.  
- Generated embeddings for each chunk using a HuggingFace model.  
- Created a **Chroma vector database** to store embeddings and metadata.  
- Implemented semantic search to retrieve relevant book recommendations.  

👏 **Congratulations on building your first vector database and semantic recommender system!**  

Keep experimenting with different embedding models, chunk sizes, and retrieval strategies to further improve your recommendations.
